# CS460 Algorithms and Their Analysis
## Programming Assignment 6: Graph algorithms -- Part 2, Topological Sort and Strongly Connected Components

**Author:** Yang Xu, Assistant Professor of Computer Science, San Diego State University

**Total points: 10**

In this assignment, you will implement the topological sort and strongly connected components algorithms.

First, you need to copy the code from previous assignment for the `Graph` class, and the `DFS()`, `DFS_visit()` functions to the following cell. **Note** that the global variable `time` should also be included.

In [51]:
class Graph:
    # constructor
    def __init__(self, directed=False):
        self.directed = directed
        self.adj = {}

    def print_graph(self):
        for i in self.adj:
            print(i, " : ", " -> ".join([str(j) for j in self.adj[i]]))

    def add_edge(self, from_vertex, to_vertex):
        ### START YOUR CODE ###
        if from_vertex in self.adj:
            if to_vertex not in self.adj[from_vertex]:
                self.adj[from_vertex].append(to_vertex)
        else:
            self.adj[from_vertex] = [to_vertex]
        if to_vertex not in self.adj:
            self.adj[to_vertex] = [] # Also add to_vertex to self.adj, but its list should be empty

        if not self.directed:
            # Flip from_vertex and to_vertex and add them to self.adj in a similar way as the block of code above
            if from_vertex in self.adj:
                if to_vertex not in self.adj[from_vertex] or from_vertex not in self.adj[to_vertex]:
                    self.add_edge(to_vertex, from_vertex)
        ### END YOUR CODE ###

In [52]:
time = 0 # Initialize the global time

def DFS(graph):
    ### START YOUR CODE ###
    visited = dict.fromkeys(graph.adj, False) # Hint: a dict whose keys are all the vertices in the graph, and values are initialized to False
    discovered = dict.fromkeys(graph.adj) # Hint: same keys as above, values initialized to None
    finished = dict.fromkeys(graph.adj) # Hint: same as above
    ### END YOUR CODE ###

    global time # Make sure to use the global variable
    time = 0
    ### START YOUR CODE ###
    for u in graph.adj: # Specify loop range
        if visited[u] == False: # Add necessary code
            DFS_visit(graph, u, visited, discovered, finished)
    ### END YOUR CODE ###

    return discovered, finished

def DFS_visit(graph, vertex, visited, discovered, finished):
    ### START YOUR CODE ###
    global time
    time += 1 # Add necessary code
    visited[vertex] = True
    discovered[vertex] = time
    for v in graph.adj[vertex]: # Specify loop range
        if visited[v] == False: # Add necessary code
            DFS_visit(graph, v, visited, discovered, finished)
    time += 1 
    finished[vertex] = time
    ### END YOUR CODE ###

---

## Task 1. Topological sort
**Points: 2**

Implement the `topological_sort()` function. First, call `DFS()` on the graph and record the **finished** time for all vertices. Next, sort the keys of `finished` dict by its values.

Note that the expected output is not fixed, i.e., there can be multiple valid results for topological sort.

*Hint*: you can use the built-in `sorted()` function and specify the `key` parameter for sorting.

In [53]:
def topological_sort(graph):
    ### START YOUR CODE ###
    _, finished = DFS(graph) # Call DFS()
    sorted_vertices = sorted(finished.items(), key=lambda kv:(kv[1])) # Sort vertices based on their finished times
    sorted_vertices = [x[0] for x in sorted_vertices]
    ### END YOUR CODE ###
    return sorted_vertices

Next, build the directed graph as shown in the figure (a) on page 613 in text book, and apply topological sort on it.

<img src="topological_sort.png">

In [54]:
# Construct the graph and apply topological sort
graph = Graph(directed=True)

### START YOUR CODE ###
graph.add_edge('undershorts', 'pants') # Example
graph.add_edge('pants', 'belt')
graph.add_edge('belt', 'jacket')
graph.add_edge('shirt', 'belt')
graph.add_edge('shirt', 'tie')
graph.add_edge('tie', 'jacket')
graph.add_edge('socks', 'shoes')
graph.add_edge('pants', 'shoes')
graph.add_edge('undershorts', 'shoes') # Add all the edges

### END YOUR CODE ###

# Do not change the code below
# Add `watch` manually
graph.adj['watch'] = []

print('Original graph:')
graph.print_graph()

print()
print('Sorted vertices:')
print(topological_sort(graph))

Original graph:
undershorts  :  pants -> shoes
pants  :  belt -> shoes
belt  :  jacket
jacket  :  
shirt  :  belt -> tie
tie  :  jacket
socks  :  shoes
shoes  :  
watch  :  

Sorted vertices:
['jacket', 'belt', 'shoes', 'pants', 'undershorts', 'tie', 'shirt', 'socks', 'watch']


**Expected output**

Original graph:\
undershorts  :  pants -> shoes\
pants  :  belt -> shoes\
belt  :  jacket\
jacket  :  \
shirt  :  belt -> tie\
tie  :  jacket\
socks  :  shoes\
shoes  :  \
watch  :

Sorted vertices:\
['jacket', 'belt', 'shoes', 'pants', 'undershorts', 'tie', 'shirt', 'socks', 'watch']

---

## Task 2. Strongly connected components
**Points: 8**

Implement the algorithm for finding the strongly connected components of a directed graph.

First, define the function to transpose a graph, that is, to copy all the vertices and reverse all the edges' direction, and return it as a new graph.

(**2 points** for this part)

In [55]:
# Transpose a directed graph: 2 points
def transpose_graph(graph):
    transposed = Graph(directed=True) # Initialize a new graph

    for vertex in graph.adj:
        ### START YOUR CODE ###
        transposed.adj[vertex] = [] # Initialize all adjacency lists to empty
        ### END YOUR CODE ###

    for vertex, adjacent_vertices in graph.adj.items():
        ### START YOUR CODE ###
        # Use a loop to append edges to transposed
        for i in adjacent_vertices:
            transposed.add_edge(i, vertex)
        ### END YOUR CODE ###

    return transposed

In [56]:
# Do not change the test code here
graph2 = Graph(directed=True)
graph2.adj = {
    'a': ['b'], 'b': ['c', 'e', 'f'], 'c': ['d', 'g'], 'd': ['c', 'h'],
    'e': ['a', 'f'], 'f': ['g'], 'g': ['f', 'h'], 'h': ['h']
}

graph2_transposed = transpose_graph(graph2)
graph2_transposed.print_graph()

a  :  e
b  :  a
c  :  b -> d
d  :  c
e  :  b
f  :  b -> e -> g
g  :  c -> f
h  :  d -> g -> h


**Expected output**

a  :  e\
b  :  a\
c  :  b -> d\
d  :  c\
e  :  b\
f  :  b -> e -> g\
g  :  c -> f\
h  :  d -> g -> h

---

Next, define the function `find_component()`, which conducts an *implicit* depth-first search that returns all the vertices contained in a strongly connected component.

Note that in this function we DO NOT use the previously defined `DFS()` function, but rather we use a recursive version of DFS that append each visited vertex to a `component` list and returns it.

(**3 points** for this part)

In [57]:
def find_component(graph, v, visited):
    visited[v] = True
    component = [v]

    ### START YOUR CODE ###
    for u in graph.adj[v]: # Iterate through all adjacent vertices of v
        if not visited[u]:
            component += find_component(graph, u, visited) # Recursive call
    ### END YOUR CODE ###

    return component

In [58]:
# Do not change the test code here
visited = {v: False for v in graph2.adj}
component = find_component(graph2_transposed, 'a', visited)
print(component)

['a', 'e', 'b']


**Expected output**
['a', 'e', 'b']

---

Finally, integrate the above defined function into `strongly_connected_components()`, which finds all the strongly components of a graph, and return them as a list of components, where each component is a list of vertices.

Within the function, we first call `topological_sort()` on the graph and return `sorted_vertices`, in which all vertices are sorted in the increase order of their finished time. So in the next step we need to call `find_component()` on each element in `sorted_vertices` in reversed order.

(**3 points** for this part)

In [59]:
def strongly_connected_components(graph):
    visited = {v: False for v in graph.adj}

    ### START YOUR CODE ###
    sorted_vertices = topological_sort(graph) # Call topological sort
    graph_transposed = transpose_graph(graph) # Transpose the graph
    ### END YOUR CODE ###

    SCCs = []
    ### START YOUR CODE ###
    for v in reversed(sorted_vertices): # Specify the order of iteration through sorted_vertices
        if not visited[v]:
            comp = find_component(graph_transposed, v, visited) # Call find_component()
            SCCs.append(comp)
    ### END YOUR CODE ###

    return SCCs

In [60]:
# Do not change the test code here
SCCs = strongly_connected_components(graph2)
print(SCCs)

[['a', 'e', 'b'], ['c', 'd'], ['g', 'f'], ['h']]


**Expected output**
[['a', 'e', 'b'], ['c', 'd'], ['g', 'f'], ['h']]